In [1]:
import torch
from PIL import Image
import requests
import torchvision.transforms as T
import matplotlib.pyplot as plt
from collections import defaultdict
import torch.nn.functional as F
import numpy as np
from skimage.measure import find_contours

from matplotlib import patches,  lines
from matplotlib.patches import Polygon

torch.set_grad_enabled(False);

In [2]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [4]:
import inflect
stemmer = inflect.engine()

In [5]:
gqa_val_q=json.load(
    open(os.path.join(GQA_PATH,'original','val_balanced_questions.json')
         ,'r')
)
print (len(gqa_val_q))
names=list(gqa_val_q.keys())

val_graphs=json.load(
    open(os.path.join(GQA_PATH,'val_sceneGraphs.json')
         ,'r')
)
print (len(val_graphs))

132062
10696


In [6]:
import skimage
from skimage import io as skimage_io
from skimage import transform as skimage_transform

def get_tokens(text_queries):
    tokenized_queries = np.array([
        module.tokenize(q, config.dataset_configs.max_query_length)
        for q in text_queries
    ])
    # Pad tokenized queries to avoid recompilation if number of queries changes:
    tokenized_queries = np.pad(
        tokenized_queries,
        pad_width=((0, 100 - len(text_queries)), (0, 0)),
        constant_values=0)
    return tokenized_queries

def get_gqa_feat(img_id):
    # Load example image:
    filename = os.path.join(GQA_PATH,'images',img_id+'.jpg')
    image_uint8 = skimage_io.imread(filename)
    image = image_uint8.astype(np.float32) / 255.0

    # Pad to square with gray pixels on bottom and right:
    h, w, _ = image.shape
    size = max(h, w)
    image_padded = np.pad(
        image, ((0, size - h), (0, size - w), (0, 0)), constant_values=0.5)

    # Resize to model input size:
    input_image = skimage.transform.resize(
        image_padded,
        (840, 840),
        anti_aliasing=True)
    return input_image

In [7]:
import clip

CUDA_DEVICE=12
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))
#the default ipykernel links to the first conda environment

model, postprocessor = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True)
model = model.to(device)
model.eval();

clip_model, preprocess = clip.load("ViT-B/32",device=device)

Using cache found in /root/.cache/torch/hub/ashkamath_mdetr_main
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def plot_inference(im, caption, plot=True):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0).cuda()

    # propagate through the model
    memory_cache = model(img, [caption], encode_and_save=True)
    outputs = model(img, [caption], encode_and_save=False, memory_cache=memory_cache)
    #print (outputs['pred_logits'])

    # keep only predictions with 0.7+ confidence
    probas = 1 - outputs['pred_logits'].softmax(-1)[0, :, -1].cpu()
    keep = (probas > 0.7).cpu()

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'].cpu()[0, keep], im.size)

    # Extract the text spans predicted by each box
    positive_tokens = (outputs["pred_logits"].cpu()[0, keep].softmax(-1) > 0.1).nonzero().tolist()
    predicted_spans = defaultdict(str)
    for tok in positive_tokens:
        item, pos = tok
        if pos < 255:
            span = memory_cache["tokenized"].token_to_chars(0, pos)
            predicted_spans [item] += " " + caption[span.start:span.end]

    labels = [predicted_spans [k] for k in sorted(list(predicted_spans .keys()))]
    if plot:
        plot_results(im, probas[keep], bboxes_scaled, labels)
    return probas[keep], bboxes_scaled.tolist(), labels

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
        masks = [None for _ in range(len(scores))]
    assert len(scores) == len(boxes) == len(labels) == len(masks)
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
            continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=c)
            ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()


def add_res(results, ax, color='green'):
    #for tt in results.values():
    if True:
        bboxes = results['boxes']
        labels = results['labels']
        scores = results['scores']
        #keep = scores >= 0.0
        #bboxes = bboxes[keep].tolist()
        #labels = labels[keep].tolist()
        #scores = scores[keep].tolist()
    #print(torchvision.ops.box_iou(tt['boxes'].cpu().detach(), torch.as_tensor([[xmin, ymin, xmax, ymax]])))
    
    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        print(text)
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

In [9]:
def vis_bbox_only(bbox,scores,input_image,gt_bbox=None,text=None,threshold=0.2,vis_pred=True):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(input_image, extent=(0, 1, 1, 0))
    ax.set_axis_off()
    if vis_pred:
        for i,box in enumerate(bbox):
            score=scores[i]
            if score<threshold:
                continue
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'r')
            ax.text(
                cx - w / 2,
                cy + h / 2 + 0.015,
                f'{score:1.2f}',
                ha='left',
                va='top',
                color='red',
                bbox={
                    'facecolor': 'white',
                    'edgecolor': 'red',
                    'boxstyle': 'square,pad=.3'
                })
    if text is not None:
        ax.set_title(text,  fontsize=12)
    if gt_bbox is not None:
        for i,box in enumerate(gt_bbox):
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'b')

In [10]:
def transform_coord(bbox,img_id):
    width=val_graphs[img_id]['width']
    height=val_graphs[img_id]['height']
    size = max(height, width)
    x=bbox[0]
    y=bbox[1]
    w=bbox[2]-bbox[0]
    h=bbox[3]-bbox[1]
    #print (bbox)
    return [(x+w/2)/size,(y+h/2)/size,w/size,h/size]

In [11]:
def preprocess_mdert_result(scores,bboxs,labels,obj,img_id):
    sf_s=[]
    sf_bbox=[]
    labels=[''.join(label.split(' ')) for label in labels]
    flag=False
    if obj in labels:
        flag=True
    #print (obj,labels)
    for i,label in enumerate(labels):
        if flag:
            if obj==label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
        else:
            if obj in label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
    #print (len(sf_s))
    if len(sf_bbox)==0:
        for i,label in enumerate(labels):
            sf_s.append(scores[i])
            sf_bbox.append(bboxs[i])
    max_s=max(sf_s)
    max_id=sf_s.index(max_s)
    return_bbox=sf_bbox[max_id]
    coord=transform_coord(return_bbox,img_id)
    return max_s.item(),coord
#only consider one bbox now

In [12]:
def generate_phrase_ground(img_id,cap,obj):
    cap=cap+' '
    im=Image.open(os.path.join(GQA_PATH,'images',img_id+'.jpg'))
    try:
        scores,bboxs,labels=plot_inference(im, cap, plot=False)
    except:
        print ('Invalid caption generation for',img_id,obj,cap)
        scores=[torch.Tensor([0.99])]
        bboxs=[[0.5,0.5,0.99,0.99]]
        labels=[obj]
    if len(bboxs)==0:
        max_s=0.99
        valid_bbox=[0.5,0.5,0.99,0.99]
    else:
        # print (scores,bboxs)
        max_s,valid_bbox=preprocess_mdert_result(scores,bboxs,labels,obj,img_id)
    return max_s,valid_bbox

def bbox_generator(layout,idx,img_id,ques_id):
    if layout[idx]['operation']=='select':
        #print (idx)
        try:
            scenic_result=load_pkl(os.path.join(GQA_PATH,
                                                'meta_one-all-scenic',
                                                ques_id+'.pkl'))
            obj=layout[idx]['argument'][0]
            scores=[s['score'] for s in scenic_result[obj]]
            bboxs=[s['bbox'] for s in scenic_result[obj]]
        except:
            bboxs=[]
            obj=layout[idx]['argument'][0]
        if len(bboxs)>0:
            max_s=max(scores)
            max_idx=scores.index(max_s)
            bbox=bboxs[max_idx]
            cap=obj
        else:
            cap=obj
            split_obj=''.join(obj.split(' '))
            #print (cap,split_obj)
            max_s,bbox=generate_phrase_ground(img_id,cap,split_obj)
            #print (max_s,bbox)
    else:
        cap,obj=cap_generator(layout,idx,img_id)
        max_s,bbox=generate_phrase_ground(img_id,cap,obj)
    return max_s,bbox,cap        
            
def cap_generator(layout,idx,img_id):
    words=[]
    dep=layout[idx]['dependencies']
    word=layout[idx]['argument'][0]
    words.append(word)
    obj=''.join(word.split(' '))
    while len(dep)>0:
        cur_step=layout[dep[0]]
        if cur_step['operation']=='relocate':
            relo_symbol=cur_step['argument'][1]
            cur_phrase=' '.join(words)
            words=[]
            words.append(cur_phrase)
            #print(cur_step)
            words.append(cur_step['argument'][0])
            dep=cur_step['dependencies']
            cur_step=layout[dep[0]]
            dep=cur_step['dependencies']
            arg=cur_step['argument'][0]
            words.append(arg)
            if relo_symbol=='o':
                words=list(reversed(words))
        else:
            arg=cur_step['argument'][0]
            words.append(arg)
            dep=cur_step['dependencies']
    cap=' '.join(words)
    return cap,obj

In [13]:
def phrase_bbox_generator_exist(layout,idx,img_id):
    cap=cap_generator_exist(layout,idx,img_id)
    max_s,bbox=generate_phrase_ground(img_id,cap,cap)
    return bbox,cap

In [14]:
def get_cropped(bb,input_image,showing=False):
    width=bb[2]*840
    height=bb[3]*840
    xy=(bb[0]*840-width/2,bb[1]*840-height/2)
    h0=int(max(0,xy[1]))
    w0=int(max(0,xy[0]))
    h1=int(max(0,840-(height+xy[1])))
    w1=int(max(0,840-(width+xy[0])))
    cropped=skimage.util.crop(input_image,((h0,h1),(w0,w1),(0,0)), copy=False)
    trans_crop=Image.fromarray(np.uint8(cropped*255.0))
    if showing:
        display.display(trans_crop)
    return trans_crop

def clip_aware_pred(img_feat,valid_bbox,candidates,showing=False):
    #tokens=clip.tokenize(candidates)
    trans_crop=get_cropped(valid_bbox,img_feat)
    if showing:
        display.display(trans_crop)
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
        text_features = clip_model.encode_text(clip.tokenize(candidates).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    #print (indices[0][0].item())
    ans=candidates[indices[0][0].item()]
    return ans

In [15]:
position_holder=['left','right','bottom','top','front']

In [16]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [17]:
def get_nouns(ques):
    nouns=[]
    tokens_tag = pos_tag(ques.replace('?','').split(' '))
    for w,pos in tokens_tag:
        if pos in ['NN','NNS'] and w not in position_holder:
            nouns.append(lemmatizer.lemmatize(w))
    return nouns

def verify_noun(nouns,obj):
    if obj in nouns or \
    lemmatizer.lemmatize(obj) in nouns or\
    stemmer.singular_noun(obj) in nouns:
        return True
    elif len(obj.split(' '))>1:
        words=obj.split(' ')
        for w in words:
            #print (w)
            if lemmatizer.lemmatize(w) in nouns or\
            stemmer.singular_noun(w) in nouns:
                return True
    elif obj in ['he','she','they']:
        return True
    return False

In [18]:
def get_relo(layout):
    ops=[step['operation'] for step in layout]
    length=len(layout)-2
    relo_id=length
    counter=0
    rela_symbol=[]
    for i in range(len(layout)-2):
        cur_op=layout[length-i]['operation']
        if cur_op=='relocate':
            relo_id=length-i
            rela_symbol=layout[length-i]['argument']
            break
        elif cur_op=='filter':
            counter+=1#find some attributes finally, if true
    return rela_symbol,relo_id, counter

In [19]:
def cap_generator_exist(layout,idx,img_id):
    words=[]
    dep=layout[idx]['dependencies']
    while len(dep)>0:
        cur_step=layout[dep[0]]
        if cur_step['operation']=='relocate':
            relo_symbol=cur_step['argument'][1]
            cur_phrase=' '.join(words)
            words=[]
            words.append(cur_phrase)
            #print(cur_step)
            words.append(cur_step['argument'][0])
            dep=cur_step['dependencies']
            new_sent=[]
            while len(dep)>0:
                cur_step=layout[dep[0]]
                dep=cur_step['dependencies']
                arg=cur_step['argument'][0]
                new_sent.append(arg)
            words.append(' '.join(new_sent))
            if relo_symbol=='o':
                words=list(reversed(words))
        else:
            arg=cur_step['argument'][0]
            words.append(arg)
            dep=cur_step['dependencies']
    cap=' '.join(words)
    return cap

In [20]:
def relocate_involved_ans(layout,img_id,ques_id,thresh=0.2):
    nouns=get_nouns(gqa_val_q[ques_id]['question'][:-1])#remove the punctuation
    ops=[step['operation'] for step in layout]
    #must have select operation
    for m,step in enumerate(layout[:-1]):
        op=step['operation']
        if op not in ['filter','select']:
            continue
        arg=step['argument'][0]
        if arg in hold_list:
            continue
        if op=='filter':
            flag=verify_noun(nouns,arg)
            if flag==False:#not a noun
                continue
        if op=='select':
            #infos=select_info[arg]
            s,_,_=bbox_generator(layout,m,img_id,ques_id)
            if s<thresh:
                return 'no'
        else:
            try:
                filter_info=load_pkl(os.path.join(GQA_PATH,
                                              'meta_filter-related',
                                              ques_id+'.pkl'))
                infos=filter_info[arg]
                s=[info['score'] for info in infos if info['score']>=thresh]
            except:
                print('No filter result',arg,ques_id)
                continue
            
            if len(s)==0:
                return 'no'
    rela_symbol,rela_id,counter=get_relo(new_prog)    
    if counter==1:
        pred=verify_relation(layout,img_id,ques_id,rela_symbol,rela_id)
    else:
        pred=verify_attr_last_step(layout,img_id,ques_id)
    return pred

In [21]:
heur_rela=['to the left of','to the right of',
           'above','under','on top of','below',
           'beneath','underneath']
#if color on the last argument, use phrase grounding and binary color function

In [22]:
def heur_rules(bb_a,bb_b,rela):
    #whether there is bb_a rela bb_b
    if rela=='to the left of':
        if bb_a[0]<bb_b[0]:
            return 'yes'
        else:
            return 'no'
    elif rela=='to the right of':
        if bb_a[0]>bb_b[0]:
            return 'yes'
        else:
            return 'no'
    elif rela in ['above','on top of']:
        if bb_a[1]>bb_b[1]:
            return 'yes'
        else:
            return 'no'
    elif rela in [ 'under','below','beneath','underneath']:
        if bb_a[1]<bb_b[1]:
            return 'yes'
        else:
            return 'no'

In [23]:
def verify_relation(layout,img_id,ques_id,rela_symbol,rela_id,showing=False):
    relation=rela_symbol[0]
    sub_obj=rela_symbol[1]
    obj=layout[-2]['argument'][0]
    cap=cap_generator_exist(layout,len(layout)-1,img_id)
    cap=cap.replace(obj,'').replace(relation,'')
    if relation in heur_rela:
        try:
            scenic_info=load_pkl(os.path.join(GQA_PATH,
                                              'meta_filter-related',
                                              ques_id+'.pkl'))
        #print (scenic_info.keys())
            info=scenic_info[obj]
            scores=[i['score'] for i in info]
            bboxs=[i['bbox'] for i in info]
            if len(scores)==0:
                return 'no'
            bb_1=bboxs[scores.index(max(scores))]
        except:
            s,b,_=bbox_generator(layout,len(layout)-2,img_id,ques_id)
            bb_1=b
        
        if rela_id==1:
            _,bb_2,_=bbox_generator(layout,0,img_id,ques_id)
        else:
            obj=layout[rela_id-1]['argument'][0]
            _,bb_2=generate_phrase_ground(img_id,cap,obj)
        if sub_obj=='o':
            bb_a=bb_2
            bb_b=bb_1
        else:
            bb_a=bb_1
            bb_b=bb_2
        pred=heur_rules(bb_a,bb_b,relation)
        if showing:
            img_feat=get_gqa_feat(img_id)
            #print (bb_a,bb_b)
            vis_bbox_only([bb_a],[0.5],img_feat,[bb_b],img_id)
    else:
        if sub_obj=='s':
            pos=' '.join([obj,'is',relation,cap])
            neg=' '.join([obj,'is not',relation,cap])
        else:
            pos=' '.join([cap,'is',relation,obj])
            neg=' '.join([cap,'is not',relation,obj])
        #print (pos,'\n',neg,'\n',gqa_val_q[ques_id]['question'])
        pred=binary_matching(img_id,[pos,neg])
    return pred

In [24]:
def binary_matching(img_id,sents):
    tokens=clip.tokenize(sents)
    im=Image.open(os.path.join(GQA_PATH,'images',img_id+'.jpg'))
    with torch.no_grad():
        text_features=clip_model.encode_text(tokens.to(device))
        image_features = clip_model.encode_image(preprocess(im).unsqueeze(0).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    indices=torch.max(similarity,1)[1]
    #print (similarity,indices)
    similarity=similarity.squeeze()
    scores=[str(similarity[0].item()),str(similarity[1].item())]
    if indices.squeeze().item()==0:
        pred='yes'
    else:
        pred='no'
    return pred

In [25]:
def verify_attr_last_step(layout,img_id,ques_id,showing=False):
    cap=cap_generator_exist(layout,len(layout)-1,img_id)
    attr=layout[-2]['argument'][0]
    phrase=cap.replace(attr,'')
    obj=layout[-3]['argument'][0]
    _,bbox=generate_phrase_ground(img_id,phrase,obj)
    input_image=get_gqa_feat(img_id)
    pred=filter_binary_color(bbox,input_image,attr,showing)
    return pred

In [26]:
def filter_binary_color(bb,input_image,color,showing=False):
    width=bb[2]*840
    height=bb[3]*840
    xy=(bb[0]*840-width/2,bb[1]*840-height/2)
    h0=int(max(0,xy[1]))
    w0=int(max(0,xy[0]))
    h1=int(max(0,840-(height+xy[1])))
    w1=int(max(0,840-(width+xy[0])))
    cropped=skimage.util.crop(input_image,((h0,h1),(w0,w1),(0,0)), copy=False)
    trans_crop=Image.fromarray(np.uint8(cropped*255.0))
    flag=True #color is a positive statement
    if 'not' in color:
        flag=False#reverse the pred
        words=color.split(' ')[1:]
        color=' '.join(words)
    pos_sent=color
    neg_sent='Not '+color
    tokens=clip.tokenize([neg_sent,pos_sent])
    with torch.no_grad():
        text_features=clip_model.encode_text(tokens.to(device))
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    indices=torch.max(similarity,1)[1]
    #print (similarity,indices)
    similarity=similarity.squeeze()
    scores=[str(similarity[0].item()),str(similarity[1].item())]
    if showing:
        vis_no_score([bb],input_image,'not '+color+'-'+'-'.join(scores))
    if indices.squeeze().item()==1:
        pred='yes'
    else:
        pred='no'
    if flag:
        return pred
    else:
        if pred=='yes':
            return 'no'
        else:
            return 'yes'

In [27]:
def generate_basic_ans(layout,img_id,ques_id,thresh=0.2):
    arg=layout[0]['argument'][0]
    scenic_result=load_pkl(os.path.join(GQA_PATH,'meta_one-all-scenic',ques_id+'.pkl'))
    info=scenic_result[arg]
    scores=[]
    scores=[i['score'] for i in info if i['score']>=thresh]
    if len(scores)==0:
        return 'no'
    if max(scores)>=thresh:
        return 'yes'
    else:
        return 'no'

In [28]:
def filter_position(bb,pos,input_image,showing=False):
    if showing:
        vis_no_score([bb],input_image,pos)
    if pos=='left':
        if bb[0]<0.4:
            return 'yes'
        else:
            return 'no'
    elif pos=='right':
        if bb[0]>0.6:
            return 'yes'
        else:
            return 'no'
    elif pos=='top':
        if bb[1]<0.4:
            return 'yes'
        else:
            return 'no'
    elif pos=='bottom':
        if bb[1]>0.6:
            return 'yes'
        else:
            return 'no'

In [29]:
def vis_no_score(bbox,input_image,text=None):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(input_image, extent=(0, 1, 1, 0))
    ax.set_axis_off()
    for i,box in enumerate(bbox):
        cx, cy, w, h = box
        ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'r')
    if text is not None:
        ax.set_title(text,  fontsize=12)
            
def verify_attr_ans(layout,img_id,ques_id,showing=False):
    attr=layout[-2]['argument'][0]#the last filter module
    input_image=get_gqa_feat(img_id)
    if len(layout)==3:
        _,bbox,_=bbox_generator(layout,0,img_id,ques_id)
    else:
        bbox,cap=phrase_bbox_generator_exist(layout,len(layout)-2,img_id)
    if attr in ['left','right','top','bottom']:
        pred=filter_position(bbox,attr,input_image,showing)
    else:
        pred=filter_binary_color(bbox,input_image,attr,showing)
    return pred

In [30]:
def generate_scene_ans(layout,img_id,ques_id,showing=False):
    attr=layout[-2]['argument'][0]#the last filter module
    input_image=get_gqa_feat(img_id)
    bbox=[0.5,0.5,0.99,0.99]#the whole image
    pred=filter_binary_color(bbox,input_image,attr,showing)
    return pred

In [31]:
def existence_for_all(layout,img_id,ques_id):
    ops=[step['operation'] for step in layout]
    if 'filter' not in ops:
        pred=generate_basic_ans(layout,img_id,ques_id)
    elif 'relocate' not in ops:
        if layout[0]['argument'][0]=='scene':
            pred=generate_scene_ans(layout,img_id,ques_id)
        else:
            pred=verify_attr_ans(layout,img_id,ques_id)
    else:
        pred=relocate_involved_ans(layout,img_id,ques_id)
    return pred

In [32]:
meta_prog=load_pkl('../meta_generated_layout/dep_all_meta_layout.pkl')

In [33]:
#focus on exist now
random.shuffle(names)
vis=0
acc=0.0
colors=defaultdict(int)
materials=defaultdict(int)
shapes=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if new_prog[-1]['operation']!='query':
        continue
    if len(new_prog[-1]['argument'])==0:
        continue
    query_t=new_prog[-1]['argument'][0]
    if query_t=='color':
        colors[row['answer']]+=1
    elif query_t=='material':
        materials[row['answer']]+=1
    elif query_t=='shape':
        shapes[row['answer']]+=1
    vis+=1
print (vis)

68167


In [34]:
hold_list=[]
hold_list.extend([c for c in colors])
hold_list.extend([c for c in materials])
hold_list.extend([c for c in shapes])
hold_list=set(hold_list)
#print (hold_list)

In [35]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>1000:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%100==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


Invalid caption generation for 2325480 waiting   waiting person 
current accuracy: 69.0
	Invalid number 0
current accuracy: 68.5
	Invalid number 0
current accuracy: 67.66666666666666
	Invalid number 0
Invalid caption generation for 2361213 bag   bag holding man 
current accuracy: 68.5
	Invalid number 0
current accuracy: 70.19999999999999
	Invalid number 0
current accuracy: 70.0
	Invalid number 0
current accuracy: 69.85714285714286
	Invalid number 0
current accuracy: 69.125
	Invalid number 0
Invalid caption generation for 2335787 book   book on top of desk 
Invalid caption generation for 2324329 fire fire 
No filter result bike 06652937
Invalid caption generation for 2324329 bike fire to the right of bike 
Invalid caption generation for 2324329 fire fire 
current accuracy: 69.11111111111111
	Invalid number 0
current accuracy: 69.1
	Invalid number 0
1001
69.13086913086913
0


In [35]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
total={}
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    total[name]=pred
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


Invalid caption generation for 2333858 boys   boys on the side of water 
Invalid caption generation for 2319436 tray   tray above paper 
Invalid caption generation for 2412014 couch couch 
No filter result men 10767993
Check: 882 00759257
current accuracy: 69.39999999999999
	Invalid number 1
Invalid caption generation for 2326319 bowl   bowl in vegetables 
current accuracy: 69.19999999999999
	Invalid number 1
Invalid caption generation for 2324329 fire fire 
No filter result fence 06652843
Invalid caption generation for 2324329 fence fence to the left of fire 
Invalid caption generation for 2324329 fire fire 
Invalid caption generation for 2374121 grass   grass near road 
Invalid caption generation for 2391143 curtains   curtains in front of windows 
Invalid caption generation for 2328203 surfboard surfboard 
No filter result man 14988840
No filter result shorts 14988840
Invalid caption generation for 2381592 food   food inside bread 
current accuracy: 69.73333333333333
	Invalid number

current accuracy: 69.79333333333332
	Invalid number 6
current accuracy: 69.72903225806452
	Invalid number 6
Invalid caption generation for 2334149 building building 
No filter result truck 17861382
current accuracy: 69.75
	Invalid number 6
Invalid caption generation for 2373294 banana   banana in front of cucumber 
Invalid caption generation for 2348606 appliance   appliance on magnets 
current accuracy: 69.72727272727272
	Invalid number 6
Invalid caption generation for 2367894 curtains   curtains on window 
Invalid caption generation for 2324329 vehicle vehicle 
No filter result middle 06652817
No filter result bag 06652817
Invalid caption generation for 2324329 bag bag to the left of middle vehicle 
Invalid caption generation for 2324329 middle   middle vehicle 
Invalid caption generation for 2363947 sign   sign in street 
current accuracy: 69.70588235294117
	Invalid number 6
Invalid caption generation for 2329006 pillow  pillow to the left of pillow 
Check: 2412014 10768021
current 

In [36]:
pkl.dump(total,open('verify.pkl','wb'))

In [38]:
counts={0:0,1:0,2:0,3:0}
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        counts[0]+=1
    elif 'relocate' not in ops:
        if new_prog[0]['argument'][0]=='scene':
            counts[1]+=1
        else:
            counts[2]+=1
    else:
        counts[3]+=1
total=sum(counts.values())
print (total)
print (counts[0]*100.0/total,
       counts[1]*100.0/total,
       counts[2]*100.0/total,
       counts[3]*100.0/total)

27413
5.7709845693649 3.2065078612337214 29.073797103564 61.94871046583738


In [39]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        continue
    if 'relocate' in ops:
        continue    
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


Check: 2412014 10768021
current accuracy: 68.0
	Invalid number 1
Check: 2403572 17455891
current accuracy: 67.0
	Invalid number 2
Check: 2324040 12228833
current accuracy: 68.60000000000001
	Invalid number 3
current accuracy: 69.39999999999999
	Invalid number 3
current accuracy: 69.08
	Invalid number 3
Check: 2412014 10767860
current accuracy: 68.89999999999999
	Invalid number 4
current accuracy: 69.22857142857143
	Invalid number 4
Check: 882 00759248
current accuracy: 69.5
	Invalid number 5
current accuracy: 69.44444444444444
	Invalid number 5
current accuracy: 69.16
	Invalid number 5
current accuracy: 69.12727272727273
	Invalid number 5
Check: 882 00759257
current accuracy: 69.13333333333334
	Invalid number 6
Check: 2328203 14988802
Check: 2324040 12228733
current accuracy: 69.12307692307692
	Invalid number 8
Check: 2412014 10767940
Check: 2391784 04214417
current accuracy: 69.01428571428572
	Invalid number 10
Check: 2403572 17455849
current accuracy: 68.88
	Invalid number 11
current

In [42]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        continue
    if 'relocate' in ops:
        continue  
    attr=new_prog[-2]['argument'][0]
    if attr not in ['left','right','top','bottom']:
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Check: 2324040 12228733
Check: 882 00759248
current accuracy: 85.6
	Invalid number 2
current accuracy: 85.2
	Invalid number 2
current accuracy: 84.33333333333334
	Invalid number 2
current accuracy: 84.7
	Invalid number 2
current accuracy: 84.56
	Invalid number 2
Check: 2403572 17455891
Check: 2324040 12228833
2858
84.88453463960812
4


In [43]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        continue
    if 'relocate' in ops:
        continue  
    attr=new_prog[-2]['argument'][0]
    if attr in ['left','right','top','bottom']:
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

current accuracy: 64.2
	Invalid number 0
Check: 2412014 10768021
current accuracy: 62.9
	Invalid number 1
Check: 882 00759257
Check: 2403572 17455849
Check: 2328203 14988802
Check: 2391784 04214417
current accuracy: 62.0
	Invalid number 5
Check: 2412014 10767940
current accuracy: 61.75000000000001
	Invalid number 6
Check: 2412014 10767860
current accuracy: 61.480000000000004
	Invalid number 7
current accuracy: 61.46666666666667
	Invalid number 7
current accuracy: 61.6
	Invalid number 7
current accuracy: 62.025
	Invalid number 7
current accuracy: 61.84444444444445
	Invalid number 7
current accuracy: 61.68
	Invalid number 7
current accuracy: 61.81818181818181
	Invalid number 7
5991
61.60908028709732
7


In [44]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        continue
    if 'relocate' in ops:
        continue  
    if len(new_prog)!=3:
        continue
    attr=new_prog[-2]['argument'][0]
    if attr in ['left','right','top','bottom']:
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

current accuracy: 62.0
	Invalid number 0
Check: 882 00759257
Check: 2391784 04214417
Check: 2412014 10767940
current accuracy: 60.699999999999996
	Invalid number 3
current accuracy: 62.06666666666667
	Invalid number 3
Check: 2328203 14988802
current accuracy: 61.650000000000006
	Invalid number 4
current accuracy: 60.96
	Invalid number 4
Check: 2412014 10767860
current accuracy: 61.133333333333326
	Invalid number 5
current accuracy: 61.771428571428565
	Invalid number 5
Check: 2403572 17455849
current accuracy: 61.724999999999994
	Invalid number 6
Check: 2412014 10768021
current accuracy: 61.46666666666667
	Invalid number 7
current accuracy: 61.7
	Invalid number 7
5444
62.16017634092579
7


In [45]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        continue
    if 'relocate' in ops:
        continue  
    if len(new_prog)==3:
        continue
    attr=new_prog[-2]['argument'][0]
    if attr in ['left','right','top','bottom']:
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

current accuracy: 56.99999999999999
	Invalid number 0
547
56.672760511883006
0


In [40]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' not in ops:
        continue
    if 'relocate' not in ops:
        continue    
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Invalid caption generation for 2412014 office office 
No filter result cabinet 10768072
Invalid caption generation for 2324040 lamp lamp 
No filter result man 12228812
No filter result motorcycle 12228812
Invalid caption generation for 2325221 wall wall 
No filter result toilet 14115917
current accuracy: 67.80000000000001
	Invalid number 0
current accuracy: 68.2
	Invalid number 0
Invalid caption generation for 2348606 appliance   appliance on magnets 
Invalid caption generation for 882 desk desk 
No filter result tower 00759318
Invalid caption generation for 882 tower desk above tower 
Invalid caption generation for 882 desk desk 
Invalid caption generation for 2412014 couch couch 
No filter result men 10767993
current accuracy: 68.26666666666667
	Invalid number 0
Invalid caption generation for 2328203 person person 
No filter result surfboard 14988835
No filter result notebook 10780303
Invalid caption generation for 882 tower tower 
No filter result desk 00759286
No filter result moni

Invalid caption generation for 2383983 toilet   toilet in front of flowers 
Invalid caption generation for 2334149 building building 
No filter result truck 17861382
Invalid caption generation for 2349040 device   device to the right of man 
Invalid caption generation for 2325221 towel towel 
No filter result people 14116057
No filter result umbrella 14116057
Invalid caption generation for 2325221 umbrella umbrella above people to the left of towel 
Invalid caption generation for 2325221 people   people to the left of towel 
current accuracy: 68.37692307692308
	Invalid number 0
Invalid caption generation for 2324040 painting painting 
No filter result shelf 12228843
Invalid caption generation for 2324040 shelf painting to the right of shelf 
Invalid caption generation for 2324040 painting painting 
Invalid caption generation for 2324329 vehicle vehicle 
No filter result middle 06652817
No filter result bag 06652817
Invalid caption generation for 2324329 bag bag to the left of middle ve

In [35]:
vis=0
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    vis+=1
print (vis)

27413


In [37]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>32:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'relocate' in ops:
        continue
    if 'filter' not in ops:
        continue
    if new_prog[0]['argument'][0]!='scene':
        continue
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

33
75.75757575757575
0


In [36]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>1000:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'relocate' not in ops:
        continue

    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


No filter result notebook 18460693
Invalid caption generation for 2367894 curtains   curtains on window 
Invalid caption generation for 2410918 pillow  pillow to the left of pillow 
current accuracy: 69.19999999999999
	Invalid number 0
No filter result notebook 11573897
Invalid caption generation for 2377682 vehicle   vehicle near grass 
Invalid caption generation for 2368963 curtain   curtain to the right of drawer 
current accuracy: 69.19999999999999
	Invalid number 0
1001
69.13086913086913
0


In [35]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    if 'filter' in ops:
        continue
        
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

current accuracy: 85.39999999999999
	Invalid number 0
Check: 2417045 1596056
Check: 2324040 12228673
current accuracy: 85.3
	Invalid number 2
Check: 2334149 17861435
current accuracy: 85.26666666666667
	Invalid number 3
1582
84.95575221238938
3


In [44]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>2200:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Check: 2324040 12228833
Invalid caption generation for 2417045 pants pants 
No filter result man 1596118
No filter result woman 1596118
Invalid caption generation for 2417045 woman woman to the left of man wearing pants 
Invalid caption generation for 2417045 man   man wearing pants 
current accuracy: 66.60000000000001
	Invalid number 1
Invalid caption generation for 2321676 bag   bag holding man 
Invalid caption generation for 2383983 toilet   toilet in front of flowers 
current accuracy: 65.9
	Invalid number 1
Invalid caption generation for 2412014 couch couch 
No filter result men 10767993
Invalid caption generation for 2329006 pillow  pillow to the left of pillow 
Invalid caption generation for 2319436 tray   tray above paper 
Check: 2403572 17455849
current accuracy: 67.80000000000001
	Invalid number 2
Check: 2334149 17861435
current accuracy: 69.15
	Invalid number 3
2201
68.8323489323035
3


In [45]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>2200:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Invalid caption generation for 2325221 towel towel 
No filter result people 14116057
No filter result umbrella 14116057
Invalid caption generation for 2325221 umbrella umbrella above people to the left of towel 
Invalid caption generation for 2325221 people   people to the left of towel 
Invalid caption generation for 2373294 banana   banana in front of cucumber 
Invalid caption generation for 2324329 vehicle vehicle 
No filter result middle 06652817
No filter result bag 06652817
Invalid caption generation for 2324329 bag bag to the left of middle vehicle 
Invalid caption generation for 2324329 middle   middle vehicle 
current accuracy: 68.4
	Invalid number 0
Invalid caption generation for 2367894 curtains   curtains on window 
Invalid caption generation for 2383595 girl   girl to the right of box 
Invalid caption generation for 2386495 curtains   curtains to the right of chair 
Check: 2324040 12228733
Invalid caption generation for 2324329 fire fire 
No filter result fence 06652843
In

In [46]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>2200:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Check: 2324040 12228673
Invalid caption generation for 2361213 bag   bag holding man 
Invalid caption generation for 2381592 food   food inside bread 
current accuracy: 73.6
	Invalid number 1
Invalid caption generation for 2334149 building building 
No filter result bus 17861326
Invalid caption generation for 2406655 fork   fork near plate 
Invalid caption generation for 2412014 couch couch 
No filter result men 10767993
Invalid caption generation for 2412014 painting painting 
No filter result men 10768138
current accuracy: 71.39999999999999
	Invalid number 1
Check: 2412014 10767860
Invalid caption generation for 2363947 sign   sign in street 
current accuracy: 69.6
	Invalid number 2
Invalid caption generation for 2350792 bike   bike on plate 
Invalid caption generation for 2374714 car   car near people 
current accuracy: 69.55
	Invalid number 2
Invalid caption generation for 2357279 mirrors   mirrors in bathroom 
Invalid caption generation for 2331158 pillow  pillow to the left of pi

In [47]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>2200:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Invalid caption generation for 2324329 truck truck 
No filter result bag 06652709
Invalid caption generation for 2324329 bag truck to the left of bag 
Invalid caption generation for 2324329 truck truck 
current accuracy: 70.6
	Invalid number 0
Invalid caption generation for 2325221 wall wall 
No filter result toilet 14115917
Invalid caption generation for 2383595 girl   girl to the right of box 
Check: 2324040 12228733
current accuracy: 69.6
	Invalid number 1
Invalid caption generation for 2355234 sofa   sofa in front of lamp 
current accuracy: 69.73333333333333
	Invalid number 1
Invalid caption generation for 2321676 bag   bag holding man 
Invalid caption generation for 2403401 counter   counter in front of cutting 
current accuracy: 70.55
	Invalid number 1
2201
70.01363016810541
1


In [48]:
#focus on exist now
#without the scene related questions

thresh=0.2
random.shuffle(names)
vis=0
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>2200:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    ops=[step['operation'] for step in new_prog]
    try:
        pred=existence_for_all(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
        print ('Check:',row['imageId'],name)
    #pred=existence_for_all(new_prog,row['imageId'],name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('current accuracy:',acc/vis*100.0)
        print ('\tInvalid number',len(invalid))
print (vis)
print (acc/vis*100.0)
print (len(invalid))

Check: 2391784 04214417
current accuracy: 70.6
	Invalid number 1
Invalid caption generation for 2331158 pillow  pillow to the right of pillow 
Invalid caption generation for 2361213 bag   bag holding man 
current accuracy: 69.39999999999999
	Invalid number 1
Invalid caption generation for 2403401 counter   counter in front of cutting 
Invalid caption generation for 2391143 curtains   curtains in front of windows 
Invalid caption generation for 2374121 grass   grass near road 
current accuracy: 69.19999999999999
	Invalid number 1
Invalid caption generation for 2333858 boys   boys on the side of water 
Invalid caption generation for 2324329 fire fire 
No filter result fence 06652929
Invalid caption generation for 2324329 fence fire to the right of fence 
Invalid caption generation for 2324329 fire fire 
current accuracy: 70.35
	Invalid number 1
Invalid caption generation for 2412014 couch couch 
No filter result men 10767993
2201
69.92276238073603
1


In [49]:
#focus on exist now
#without the scene related questions

thresh=0.2
#random.shuffle(names)
vis=0
acc=0.0
counts={'yes':0,'no':0}
invalid=[]
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]

    if row['types']['structural']!='verify':
        continue
    counts[row['answer']]+=1
print (counts)

{'yes': 13569, 'no': 13844}
